#**My Variant Calling Pipeline**

Each Sample in Samples is an array [0,1,2] where:
* 0 is the file name
* 1 is the forward run
* 2 is the reverse run

In [10]:
Data = {'Samples':
 [['CapturedMaleBaselineD_S1', 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR116/023/SRR11676923/SRR11676923_1.fastq.gz', 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR116/023/SRR11676923/SRR11676923_2.fastq.gz'],
 ['CapturedMaleBaselineD_S2', 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR116/019/SRR11676919/SRR11676919_1.fastq.gz', 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR116/019/SRR11676919/SRR11676919_2.fastq.gz'],
 ['EscapedMaleBaselineD_S1', 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR116/053/SRR11676853/SRR11676853_1.fastq.gz', 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR116/053/SRR11676853/SRR11676853_2.fastq.gz'],
 ['EscapedMaleBaselineD_S2', 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR116/062/SRR11676862/SRR11676862_1.fastq.gz', 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR116/062/SRR11676862/SRR11676862_2.fastq.gz']],
 'ZebrafishRefGenome': 'https://ftp.ensembl.org/pub/release-114/fasta/danio_rerio/dna/Danio_rerio.GRCz11.dna.primary_assembly.fa.gz',
 'ZebrafishRefGTF': 'https://ftp.ensembl.org/pub/release-114/gtf/danio_rerio/Danio_rerio.GRCz11.114.gtf.gz'}

## Downloading Files

In [ ]:
import requests

# Download Datasets
for sample_info in Data['Samples']:
  sample_name = sample_info[0]
  read1_url = sample_info[1]
  read2_url = sample_info[2]
  !wget -O {sample_name}_R1.fastq.gz {read1_url}
  !wget -O {sample_name}_R2.fastq.gz {read2_url}

# Download ZebrafishRefGenome
genome_url = Data['ZebrafishRefGenome']
genome_filename = 'ZebrafishRefGenome.fa.gz'
!wget -O {genome_filename} {genome_url}

# Download ZebrafishRefGTF
gtf_url = Data['ZebrafishRefGTF']
gtf_filename = 'ZebrafishRefGTF.gtf.gz'
!wget -O {gtf_filename} {gtf_url}

## Moving Files into folders

In [12]:
mkdir references raw_data FASTQC_reports

### Move zebrafishRefGTF.gtf.gz and ZebrafishRefGenome.fa.gz into references/ and then unzip it using gunzip

In [ ]:
!mv ZebrafishRefGTF.gtf.gz references/
!gunzip references/ZebrafishRefGTF.gtf.gz
!mv ZebrafishRefGenome.fa.gz references/
!gunzip references/ZebrafishRefGenome.fa.gz

### Move all files ending in "fastq.gz" into raw_data/

In [13]:
!mv *.fastq.gz raw_data/

## FastQC & MultiQC

In [ ]:
!apt update
!apt install fastqc -y
!pip install multiqc

In [ ]:
import os

input_dir = "/content/raw_data"
output_dir = "FASTQC_reports"

# Loop through all .fastq.gz files in the input directory
for dataset in Data['Samples']:
    default_file_path = os.path.join(input_dir, dataset[0])
    forward_run = default_file_path + "_R1.fastq.gz"
    reverse_run = default_file_path + "_R2.fastq.gz"

    !fastqc {forward_run} {reverse_run} -o {output_dir}

In [ ]:
!multiqc FASTQC_reports/

## Fastp

In [ ]:
!wget http://opengene.org/fastp/fastp
!chmod a+x fastp
!mv fastp /usr/local/bin/

In [21]:
mkdir trimmed_data fastp_reports

In [ ]:
import os

input_dir = "/content/raw_data"
output_dir = "/content/trimmed_data"
fastp_reports_dir = "/content/fastp_reports"

# Loop through all .fastq.gz files in the input directory
for dataset in Data['Samples']:
    default_file_path = os.path.join(input_dir, dataset[0])
    output_file_path = os.path.join(output_dir, dataset[0])
    fastp_reports_file_path = os.path.join(fastp_reports_dir, dataset[0])

    forward_run = default_file_path + "_R1.fastq.gz"
    reverse_run = default_file_path + "_R2.fastq.gz"

    trimmed_forward_run = output_file_path + "_trimmed_R1.fastq.gz"
    trimmed_reverse_run = output_file_path + "_trimmed_R2.fastq.gz"

    fastp_report_html = fastp_reports_file_path + "_fastp_report.html"
    fastp_report_json = fastp_reports_file_path + "_fastp_report.json"

    !fastp \
      -i {forward_run} \
      -I {reverse_run} \
      -o {trimmed_forward_run} \
      -O {trimmed_reverse_run} \
      -q 20 \
      -u 30 \
      -n 5 \
      -l 36 \
      --detect_adapter_for_pe \
      -h {fastp_report_html} \
      -j {fastp_report_json}

## Rerunning FastQC & MultiQC after Trimming

In [25]:
mkdir Post_trimming_FASTQC_reports

In [ ]:
import os

input_dir = "/content/trimmed_data"
output_dir = "Post_trimming_FASTQC_reports"

# Loop through all .fastq.gz files in the input directory
for dataset in Data['Samples']:
    default_file_path = os.path.join(input_dir, dataset[0])
    forward_run = default_file_path + "_trimmed_R1.fastq.gz"
    reverse_run = default_file_path + "_trimmed_R2.fastq.gz"

    !fastqc {forward_run} {reverse_run} -o {output_dir}

In [ ]:
!multiqc Post_trimming_FASTQC_reports/

## Aligning using Minimap2

In [ ]:
!apt update
!apt install -y minimap2

In [27]:
mkdir BAMs

### Creating reference Genome Index

In [ ]:
!minimap2 -d references/ZebrafishRefGenome.mmi references/ZebrafishRefGenome.fa

### Aligning

In [ ]:
import os

input_dir = "/content/trimmed_data"
output_dir = "/content/BAMs"
ref_genome_index_path = "references/ZebrafishRefGenome.mmi"

# Loop through all .fastq.gz files in the input directory
for dataset in Data['Samples']:
    default_file_path = os.path.join(input_dir, dataset[0])
    forward_run = default_file_path + "_trimmed_R1.fastq.gz"
    reverse_run = default_file_path + "_trimmed_R2.fastq.gz"
    output_file_path = os.path.join(output_dir, dataset[0]) + '.sam'

    !minimap2 -ax sr {ref_genome_index_path} {forward_run} {reverse_run} > {output_file_path}

## Sorting and Indexing using samtools

In [ ]:
!apt install -y samtools

### Creating reference Genome Index

In [ ]:
!samtools faidx /content/references/ZebrafishRefGenome.fa

### Converting .sam files to .bam

In [ ]:
import os

input_dir = "/content/BAMs"
output_dir = "/content/BAMs"

# Loop through all .fastq.gz files in the input directory
for dataset in Data['Samples']:
    input_file = os.path.join(input_dir, dataset[0]) + ".sam"
    output_file = os.path.join(output_dir, dataset[0]) + '.bam'

    !samtools view -@ 2 -bS {input_file} > {output_file}

### Sorting

In [ ]:
import os

input_dir = "/content/BAMs"
output_dir = "/content/BAMs"

# Loop through all .fastq.gz files in the input directory
for dataset in Data['Samples']:
    input_file = os.path.join(input_dir, dataset[0]) + ".bam"
    output_file = os.path.join(output_dir, dataset[0]) + '_Sorted.bam'

    !samtools sort -@ 2 -o {output_file} {input_file}

### Indexing

In [ ]:
import os

input_dir = "/content/BAMs"

# Loop through all .fastq.gz files in the input directory
for dataset in Data['Samples']:
    input_file = os.path.join(input_dir, dataset[0]) + '_Sorted.bam'

    !samtools index {input_file}

## Variant Calling using freebayes

In [ ]:
!apt update
!apt install -y freebayes

In [34]:
mkdir VCFs

In [ ]:
import os

input_dir = "/content/BAMs"
output_dir = "/content/VCFs"
ref_Genome = "references/ZebrafishRefGenome.fa"

# Loop through all .fastq.gz files in the input directory
for dataset in Data['Samples']:
    input_file = os.path.join(input_dir, dataset[0]) + "_Sorted.bam"
    output_file = os.path.join(output_dir, dataset[0]) + '.vcf'

    !freebayes -f {ref_Genome} {input_file} > {output_file}

## VCF Filtering using vcf filter

### Installing Miniconda

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

### Installing vcf filter

In [ ]:
!conda install -c bioconda vcflib

### Running vcf filter

In [ ]:
import os

input_dir = "/content/VCFs"
output_dir = "/content/VCFs"

# Loop through all .fastq.gz files in the input directory
for dataset in Data['Samples']:
    input_file = os.path.join(input_dir, dataset[0]) + ".vcf"
    output_file = os.path.join(output_dir, dataset[0]) + '_filtered.vcf'

    !vcffilter -f "QUAL > 20" {input_file} > {output_file}

## - Variant Effect Predictor (VEP) was done on usegalaxy.eu